# Let's Do Some Video Analysis

What better way to learn a concept than to get your hands dirty with code. So let us pick up a sample problem and see how can we do some basic image processing to do some simple analytics on a video. 

We will implement a method to achieve a quick and dirty solution for a problem but we will discuss various ways through which the problem can be attacked to broaden our understanding of how to build different candidate solutions for a problem and understand/analyse the pros and cons of each of those approaches. 

Some of the major factors which we will discuss are **FPS Delivered** and **Compute Power** as these are probably the most important factors as far as the deployability of a video analysis algorithm is considered.

**Some Ineteresting Reads**:

- [Video Analysis using Opencv-Python](https://people.revoledu.com/kardi/tutorial/Python/Video+Analysis+using+OpenCV-Python.html#:~:text=Video%20Analysis%20using%20OpenCV%2DPython&text=This%20tutorial%20is%20a%20practice,numpy%20and%20math%20modules%20installed.)
- [Video Analysis - OpenCV Tutorial](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_table_of_contents_video/py_table_of_contents_video.html)

## Example Problems

We have a stock video of a person juggling 3 multicolored balls. The sample problems which we can make from this video are:

- Identifying the balls as they juggle
- Tracking the balls
- Calculate frequency of each ball (1/Time between one juggle)

**Video Ref**: https://www.pexels.com/video/person-juggling-balls-854421/

## Let us see the video and process it

In [513]:
# Importing the libraries

import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt

In [514]:
# Reading the video file

vs = cv2.VideoCapture('..//assets//videos//juggling.mp4')

### Control Function

We put the visualization code inside a control function so that we can reuse it during experimentation. This function is used to control how we want to process the video. We also add resizing and processing capability by adding _frac_ and _func_ arguments.

- _frac_ : fraction to resize video frame. Defaults to 0.2. This is useful for high resolution videos.
- _func_ : func to be used to process video. Defaults to None. If not passed then frame is show as it is.

In [515]:
# Defining function to process and visualize given video

def processAndShow(vs, func = None, frac = 0.2):
    seconds = 0
    count = 0
    
    while vs.isOpened():
        ret, fr = vs.read()
        
        if ret == False:
            break
        
        tic = time.time()
        fr = cv2.resize(fr, (int(vs.get(cv2.CAP_PROP_FRAME_WIDTH)*frac), 
                             int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT)*frac)), fx = frac, fy = frac)
        if func is not None:
            fr = func(fr)
        toc = time.time()
        
        count+=1
        seconds+=toc-tic
        
        if seconds < 1:
            cv2.putText(fr, 'FPS: {0:.2f}'.format(count), (10, 30),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
        else:
            cv2.putText(fr, 'FPS: {0:.2f}'.format(count/seconds), (10, 30),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
            
        cv2.imshow('Output Feed', fr)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    vs.release()
    cv2.destroyAllWindows()

In [516]:
# Calling the control function without func for only visualization and no processing

# processAndShow(vs)

### Processing Function

We add the analysis code to a processing function and pass it as an argument to our controlling function. This is a simple way to decouple the control and processing logic. 

- _fr_ : This function should have frame as a **mandatory argument**. The control function should pass each frame to this function.
- Any objects to be used in the processing function can be added as default arguments to this function

In [517]:
# Processing function definition

def proc(fr, backSub = cv2.createBackgroundSubtractorMOG2()):
    
    # Step 1. Remove the background information. That is only keep the moving pixels.
    fr_ = backSub.apply(fr)
    
    # Step 2. Apply Morphological Erosion/Dilation with circular/elliptical kernels to refine circular blobs
    fr_ = cv2.erode(fr_, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 5)
    fr_ = cv2.dilate(fr_, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 5)
        
    # Step 3. Extract contours
    contours, hierarchy = cv2.findContours(fr_, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        contours_poly = cv2.approxPolyDP(c, 3, True)
        center, radius = cv2.minEnclosingCircle(contours_poly)
        
        # Step 4. Filter the contours based on the area, using radius as proxy for area
        if radius > 40 and radius < 50:
            cv2.circle(fr, (int(center[0]), int(center[1])), int(radius), [255, 0, 0], thickness = 10)

    return fr

In [518]:
processAndShow(vs, proc)